<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/transfer%20learning%20fine-tuning/function%20documentation%20generation/ruby/large_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h3>Predict the documentation for ruby code using codeTrans transfer learning finetuning model</h3>**
<h4>You can make free prediction online through this 
<a href="https://huggingface.co/SEBIS/code_trans_t5_large_code_documentation_generation_ruby_transfer_learning_finetune">Link</a></h4> (When using the prediction online, you need to parse and tokenize the code first.)

**1. Load necessry libraries including huggingface transformers**

In [1]:
!pip install -q transformers sentencepiece

     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 1.2MB 59.8MB/s 
     |████████████████████████████████| 3.2MB 61.9MB/s 
     |████████████████████████████████| 890kB 55.9MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

**2. Load the token classification pipeline and load it into the GPU if avilabile**

In [3]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_ruby_transfer_learning_finetune"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_ruby_transfer_learning_finetune", skip_special_tokens=True),
    device=0
)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:970: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


**3 Give the code for summarization, parse and tokenize it**

In [4]:
code = "def add(severity, progname, &block)\n      return true if io.nil? || severity < level\n      message = format_message(severity, progname, yield)\n      MUTEX.synchronize { io.write(message) }\n      true\n    end" #@param {type:"raw"}

In [5]:
!pip install tree_sitter
!git clone https://github.com/tree-sitter/tree-sitter-ruby

     |████████████████████████████████| 112kB 2.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.2.2-cp36-cp36m-linux_x86_64.whl size=297843 sha256=1afc04fefc275739aef45e85df292acaa1b8c58f40bcedf563fb842168c694e2
  Stored in directory: /root/.cache/pip/wheels/2b/c0/0f/32967a2a849330fe51b835c77efae3f83b903a40f5132102d3
Successfully built tree-sitter
Cloning into 'tree-sitter-ruby'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 9517 (delta 18), reused 17 (delta 7), pack-reused 9475
Receiving objects: 100% (9517/9517), 293.62 MiB | 19.89 MiB/s, done.
Resolving deltas: 100% (4202/4202), done.


In [6]:
from tree_sitter import Language, Parser

Language.build_library(
  'build/my-languages.so',
  ['tree-sitter-ruby']
)

RUBY_LANGUAGE = Language('build/my-languages.so', 'ruby')
parser = Parser()
parser.set_language(RUBY_LANGUAGE)

In [7]:
def get_string_from_code(node, lines):
  line_start = node.start_point[0]
  line_end = node.end_point[0]
  char_start = node.start_point[1]
  char_end = node.end_point[1]
  if line_start != line_end:
    code_list.append(' '.join([lines[line_start][char_start:]] + lines[line_start+1:line_end] + [lines[line_end][:char_end]]))
  else:
    code_list.append(lines[line_start][char_start:char_end])

def my_traverse(node, code_list):
  lines = code.split('\n')
  if node.child_count == 0:
    get_string_from_code(node, lines)
  elif node.type == 'string':
    get_string_from_code(node, lines)
  else:
    for n in node.children:
      my_traverse(n, code_list)
 
  return ' '.join(code_list)

In [8]:
tree = parser.parse(bytes(code, "utf8"))
code_list=[]
tokenized_code = my_traverse(tree.root_node, code_list)
print("Output after tokenization: " + tokenized_code)

Output after tokenization: def add ( severity , progname , & block ) return true if io . nil? || severity < level message = format_message ( severity , progname , yield ) MUTEX . synchronize { io . write ( message ) } true end


**4. Make Prediction**

In [9]:
pipeline([tokenized_code])

[{'summary_text': 'Writes a message to the log device .'}]